In [1]:
# Load libraries
import numpy
from matplotlib import pyplot
import matplotlib.pyplot as plt
from pandas import read_csv
import pandas as pd
from pandas import set_option
from pandas.plotting import scatter_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss
from sklearn.metrics import roc_auc_score,roc_curve
import seaborn as sns
%matplotlib inline
from pickle import dump
from pickle import load
import warnings
warnings.filterwarnings("ignore")

#''''''''''''''''''''''''''''''''''''''''''
import h2o
import time
import os
from h2o.estimators.random_forest import H2ORandomForestEstimator
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.grid import H2OGridSearch
from h2o.automl import H2OAutoML
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
h2o.init()


# DATA PREPARATION

In [ ]:
#dataset=pd.read_csv('labor1.csv')
#dataset=pd.read_csv('labor1.csv',na_values='?')
#dataset = dataset.replace('?', numpy.NaN)
#dataset=pd.read_csv('sonar.csv',header=None,skiprows=1)
#--------------------------------------------------------
#dataset=pd.read_excel('labor1.xlsx')
#dataset=pd.read_excel('labor1.xlsx',na_values='?')
#dataset = dataset.replace('?', numpy.NaN)
#dataset=pd.read_excel('sonar.xlsx',header=None,skiprows=1)
#--------------------------------------------------------
#names=['age','menopause','tumor-size','inv-nodes','node-caps','deg-malig','breast','breast-quad','irradiat','Class']
#dataset=pd.read_csv('breast-cancer.csv',header=None,names=names,delim_whitespace=False)
#--------------------------------------------------------
#dataset=pd.read_csv('segment-challenge.csv',header=None,skiprows=1)

In [ ]:
dataset.head()

In [ ]:
# shape
print(dataset.shape)

In [ ]:
# types
set_option('display.max_rows', 500)
print(dataset.dtypes)

In [ ]:
# descriptions, change precision to 3 places
set_option('precision', 3)
print(dataset.describe())


In [ ]:
# descriptions, change precision to 3 places
set_option('precision', 3)
print(dataset.describe(include="object"))


In [ ]:
# class distribution
print(dataset.groupby(60).size())

# CLEANING/MISSING VALUES

In [10]:
# CHECKING OF MISSING VALUES

In [ ]:
print("Total missing values of each columns")
print(dataset.isnull().sum())
print(" ")
print("Total missing values is:")
print(dataset.isnull().any().sum())
print(" ")
print("missing values(True or False)")
print(dataset.isnull().any())

In [11]:
# REMOVAL OF MISSING VALUES

In [ ]:
total = dataset.isnull().sum().sort_values(ascending=False)
percent=(dataset.isnull().sum()*100/len(dataset)).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1,keys=['Total', 'Percent'])
missing_data.head(20)

In [ ]:
# summarize the number of unique values in each column
print(dataset.nunique())

In [ ]:
# delete columns with a single unique value
# get number of unique values for each column
counts = dataset.nunique()
# record columns to delete
to_del = [i for i,v in enumerate(counts) if v == 1]
print(to_del)
# drop useless columns
dataset.drop(to_del, axis=1, inplace=True)
print(dataset.shape)

In [ ]:
# locate rows of duplicate data

# calculate duplicates
dups = dataset.duplicated()
# report if there are any duplicates
print(dups.any())
# list all duplicate rows
print(dataset[dups])

In [ ]:
# delete rows of duplicate data from the dataset

print("actual dataset",dataset.shape)
# delete duplicate rows
dataset.drop_duplicates(inplace=True)
print("new dataset",dataset.shape)

In [ ]:
#deleting the "Unnamed: 32" column
dataset.drop("Unnamed: 32",axis=1,inplace=True)


In [ ]:
#removal of column wih higher percentage(missing value)
pct_null=dataset.isnull().sum()/len(dataset)
missing_features=pct_null[pct_null > 0.70].index
dataset.drop(missing_features,axis=1,inplace=True)

In [ ]:
# REMOVAL OF COLUMN WITH HIGHER MISSING VALUES
dataset=dataset.drop("columns",axis=1,inplace=True)

In [ ]:
#removal of missing value from any rows
dataset.dropna(inplace=True)

In [ ]:
#FOR NUMERIC
dataset.fillna(dataset.mean(), inplace=True)
dataset.fillna(dataset.median(), inplace=True)
dataset.interpolate()

In [ ]:
#FOR CATEGORIES ONLY
dataset.fillna(dataset.mode(), inplace=True)
#FOR CATEGORIES ONLY
dataset.fillna(("No"), inplace=True)

In [ ]:
#FOR NUMERIC AND CATEGORIES
dataset=dataset.fillna(method='bfill',limit=2)
dataset=dataset.fillna(method='ffill')
dataset.interpolate(mmethod='pad',limit=2)

# DATA VISUALIZATION

In [ ]:
plt.figure(figsize=(9,5))
sns.countplot(x="job",hue="y",data=dataset)
print(pd.crosstab(dataset["job"],dataset["y"]))

In [ ]:
grid=sns.FacetGrid(dataset,row="Gender",col="Married",size=2.2,aspect=1.6)
grid.map(plt.hist,"ApplicantIncome",alpha=0.5,bins=10)
grid.add_legend()

# ENCODING

In [ ]:
#MAPPING
dataset.customer_type=dataset.customer_type.map({1:'one',2:'two'})

In [ ]:
#INDIVIDUAL ENCODING
#conversion of categories value to numeric value
from sklearn.preprocessing import LabelEncoder
label=LabelEncoder()
dataset["class"]=label.fit_transform(dataset["class"])
dataset.head()

In [ ]:
#encoding for * categories to numeric value
dataset=dataset.select_dtypes(include=['object']).copy()

#-------------------------------------------------

#conversion of * categories value to numeric value

from sklearn.preprocessing import LabelEncoder
label=LabelEncoder()
dataset=dataset.apply(label.fit_transform)

#---------------------------------------------

dataset.head()

In [ ]:
# encoding for both numeric and categories(altogether,conversion of categories value to numeric value

dataset_obj=dataset.select_dtypes(include=['object']).copy()
dataset_int=dataset.select_dtypes(include=['int64']).copy()
dataset_obj.head()
dataset_int.head()

#-------------------------------------------------

from sklearn.preprocessing import LabelEncoder
label=LabelEncoder()
dataset_obj=dataset_obj.apply(label.fit_transform)
dataset_obj.head()

#----------------------------------------------

#how to join seperated column

dataset=dataset_int.join(dataset_obj)

#----------------------------------------------

dataset.head()

In [ ]:
# encoding for both numeric and categories(altogether,conversion of categories value to numeric value
dataset_float=dataset.select_dtypes(include=['float64']).copy()
dataset_obj=dataset.select_dtypes(include=['object']).copy()
dataset_int=dataset.select_dtypes(include=['int64']).copy()


#-------------------------------------------------

from sklearn.preprocessing import LabelEncoder
label=LabelEncoder()
dataset_obj=dataset_obj.apply(label.fit_transform)


#----------------------------------------------

#how to join seperated column

dataset1=dataset_int.join(dataset_float)
dataset=dataset1.join(dataset_obj)
#----------------------------------------------

dataset.head()

# VISUALIZATION

In [ ]:
#Correlation for numeric
plt.figure(figsize=(16,5))
sns.heatmap(dataset.corr(),annot=True)
plt.title("Correlation Matrix(for dataset)")

# OUTLIER

In [ ]:
dataset.shape

In [ ]:
from scipy import stats
import numpy as np

z=np.abs(stats.zscore(dataset))
dataset=dataset[(z<3).all(axis=1)]

In [ ]:
dataset.shape

In [ ]:
#conversion of types
dataset.customer_type=dataset.customer_type.astype('int64')

In [ ]:
#Correlation for numeric
plt.figure(figsize=(16,5))
sns.heatmap(dataset.corr(),annot=True)
plt.title("Correlation Matrix(for dataset)")

# conversion of types

In [ ]:
#conversion of types
dataset.customer_type=dataset.customer_type.astype('int64')


# MAPPING

In [ ]:
dataset.customer_type=dataset.customer_type.map({1:'one',2:'two'})

# Validation Dataset For Features Selection

In [ ]:
X = dataset.iloc[:,0:13] #independent columns
Y = dataset.iloc[:,-1] #target column 


# FEATURES SELECTION FOR BINARY CLASSIFICATION

In [ ]:
# Univariate Selection (Chi-squared for classification) 2
import numpy as np
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2


#apply SelectKBest class to extract top 10 best features
bestfeatures = SelectKBest(score_func=chi2, k=8)
fit = bestfeatures.fit(X,Y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)
#concat two dataframes for better visualization
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score'] #naming the dataframe columns
print(featureScores.nlargest(10,'Score')) #print 10 best features

In [ ]:
#. Feature Importance for classification
from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np


from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt
model = ExtraTreesClassifier()
model.fit(X,Y)
print(model.feature_importances_) #use inbuilt class feature_importances of tree based classifiers
#plot graph of feature importances for better visualization
feat_importances = pd.Series(model.feature_importances_,index=X.columns)
feat_importances.nlargest(10).plot(kind='barh')
plt.show()

# OVERSAMPLING/UNDERSAMPLING USING SMOTE/NEARMISS

In [ ]:
data=dataset.sample(frac=0.75,random_state=7)
data.reset_index(inplace=True,drop=True)
unseen=dataset.drop(data.index)
unseen.reset_index(inplace=True,drop=True)

In [ ]:
unseen.to_csv("unseen.csv",index=False)

In [ ]:
data.shape,unseen.shape

In [ ]:
X = data.iloc[:,0:19] #independent columns
Y = data.iloc[:,-1] #target column 

In [ ]:
sm=SMOTE(random_state=7,k_neighbors=7)
X_smote,Y_smote=sm.fit_sample(X,Y)

In [ ]:
X_smote.shape,Y_smote.shape,X.shape,Y.shape

In [ ]:
X_data=pd.DataFrame(X_smote)
Y_data=pd.DataFrame(Y_smote)
data_csv=X_data.join(Y_data)
data_csv.to_csv("data1_csv.csv",index=False)

In [ ]:
from collections import Counter

In [ ]:
print("resampling before smote{}". format(Counter(Y)))
print("resampling after smote{}". format(Counter(Y_smote)))

In [ ]:
# Test options and evaluation metric
num_folds = 10
seed = 7
scoring = 'accuracy'

# h2o

In [ ]:
#dataset_h20=h2o.H2OFrame(dataset)
#data=h2o.import_file("pima.csv")

In [ ]:
data.head()

In [ ]:
data.describe()

In [ ]:
for col in data.columns:
    data[col].hist()

In [ ]:
plt.figure(figsize=(10,10))
corr = data.cor().as_data_frame()
corr.index = data.columns
sns.heatmap(corr, annot = True, cmap='RdYlGn', vmin=-1, vmax=1)
plt.title("Correlation Heatmap", fontsize=16)
plt.show()


# Train Test validation split

In [ ]:
train, valid = data.split_frame(ratios=[0.7], seed=1234)
response = "class"
train[response] = train[response].asfactor()
valid[response] = valid[response].asfactor()

print("Number of rows in train, valid and test set : ", train.shape[0], valid.shape[0])


# Modeling Building


In [ ]:
predictors = data.columns[:-1]
gbm = H2OGradientBoostingEstimator()
gbm.train(x=predictors, y=response, training_frame=train)

In [ ]:
print(gbm)

In [ ]:
perf = gbm.model_performance(valid)
print(perf)

# Model Tuning

In [ ]:
#stopping_metric = "AUC",
gbm_tune = H2OGradientBoostingEstimator(
    ntrees = 1000,
    learn_rate = 0.01,
    stopping_rounds = 20,
    stopping_metric = "AUC",
    col_sample_rate = 0.7,
    sample_rate = 0.7,
    seed = 1234
)      
gbm_tune.train(x=predictors, y=response, training_frame=train, validation_frame=valid)

In [ ]:
gbm_tune.model_performance(valid)

# Grid Search

In [ ]:
#stopping_metric = "AUC",
from h2o.grid.grid_search import H2OGridSearch

In [ ]:
gbm_grid = H2OGradientBoostingEstimator(
    ntrees = 1000,
    learn_rate = 0.01,
    stopping_rounds = 20,
    stopping_metric = "AUC",
    col_sample_rate = 0.7,
    sample_rate = 0.7,
    seed = 1234
) 

hyper_params = {'max_depth':[4,6,8,10,12]}
grid = H2OGridSearch(gbm_grid, hyper_params,
                         grid_id='depth_grid')
#Train grid search
grid.train(x=predictors, 
           y=response,
           training_frame=train,
           validation_frame=valid)

In [ ]:
print(grid)

In [ ]:
sorted_grid = grid.get_grid(sort_by='auc',decreasing=True)
print(sorted_grid)


In [ ]:
grid.model_performance(valid)

# K-Fold cross validation:

In [ ]:
#stopping_metric = "AUC",
cv_gbm = H2OGradientBoostingEstimator(
    ntrees = 3000,
    learn_rate = 0.05,
    stopping_rounds = 20,
    stopping_metric = "AUC",
    nfolds=4, 
    seed=2018)
cv_gbm.train(x = predictors, y = response, training_frame = train, validation_frame=valid)
cv_summary = cv_gbm.cross_validation_metrics_summary().as_data_frame()
cv_summary

In [ ]:
cv_gbm.model_performance(valid).auc()

# H20 with XGBoost

In [ ]:
from h2o.estimators import H2OXGBoostEstimator

cv_xgb = H2OXGBoostEstimator(
    ntrees = 1000,
    learn_rate = 0.05,
    stopping_rounds = 20,
    stopping_metric = "AUC",
    nfolds=4, 
    seed=2018)
cv_xgb.train(x = predictors, y = response, training_frame = train, validation_frame=valid)
cv_xgb.model_performance(valid).auc()

# feature importance

In [ ]:
cv_xgb.varimp_plot()

# AutoML : Automatic Machine Learning:

In [ ]:
from h2o.automl import H2OAutoML

aml = H2OAutoML(max_models = 10, max_runtime_secs=60, seed = 1)
aml.train(x=predictors, y=response, training_frame=train, validation_frame=valid)

In [ ]:
lb = aml.leaderboard
lb


In [ ]:
dataset_pred=aml.leader.predict(valid)

In [ ]:
dataset_pred.head()

In [ ]:
aml.predict(valid)

In [ ]:
aml.leader.model_performance(valid)

In [ ]:
model_ids=list(aml.leaderboard['model_id'].as_data_frame().iloc[:,0])

In [ ]:
model_ids

# VALIDATION DATASET 

In [ ]:
# Split-out validation dataset
array = dataset.values
X = array[:,0:60].astype(float)
Y = array[:,60]
validation_size = 0.20
seed = 7
X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y,
test_size=validation_size, random_state=seed)

In [ ]:
# Test options and evaluation metric
num_folds = 10
seed = 7
scoring = 'accuracy'

# ALGORITHM CLASSIFICATION

In [ ]:
# Spot-Check Algorithms
models = []
models.append(('LR', LogisticRegression()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC()))
results = []
names = []
for name, model in models:
    kfold = KFold(n_splits=num_folds, random_state=seed)
    cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

In [ ]:
# Compare Algorithms
fig = pyplot.figure()
fig.suptitle('Algorithm Comparison')
ax = fig.add_subplot(111)
pyplot.boxplot(results)
ax.set_xticklabels(names)
pyplot.show()


# PIPELINES

In [ ]:
# Standardize the dataset
pipelines = []
pipelines.append(('ScaledLR', Pipeline([('Scaler', StandardScaler()),('LR',
LogisticRegression())])))
pipelines.append(('ScaledLDA', Pipeline([('Scaler', StandardScaler()),('LDA',
LinearDiscriminantAnalysis())])))
pipelines.append(('ScaledKNN', Pipeline([('Scaler', StandardScaler()),('KNN',
KNeighborsClassifier())])))
pipelines.append(('ScaledCART', Pipeline([('Scaler', StandardScaler()),('CART',
DecisionTreeClassifier())])))
pipelines.append(('ScaledNB', Pipeline([('Scaler', StandardScaler()),('NB',
GaussianNB())])))
pipelines.append(('ScaledSVM', Pipeline([('Scaler', StandardScaler()),('SVM', SVC())])))
results = []
names = []
for name, model in pipelines:
    kfold = KFold(n_splits=num_folds, random_state=seed)
    cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

In [ ]:
# Compare Algorithms
fig = pyplot.figure()
fig.suptitle('Scaled Algorithm Comparison')
ax = fig.add_subplot(111)
pyplot.boxplot(results)
ax.set_xticklabels(names)
pyplot.show()


# Algorithm Tuning

In [ ]:
# Tune scaled SVM
scaler = StandardScaler().fit(X_train)
rescaledX = scaler.transform(X_train)
c_values = [0.1, 0.3, 0.5, 0.7, 0.9, 1.0, 1.3, 1.5, 1.7, 2.0]
kernel_values = ['linear', 'poly', 'rbf', 'sigmoid']
param_grid = dict(C=c_values, kernel=kernel_values)
model = SVC()
kfold = KFold(n_splits=num_folds, random_state=seed)
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring=scoring, cv=kfold)
grid_result = grid.fit(rescaledX, Y_train)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
# Tune scaled KNN
scaler = StandardScaler().fit(X_train)
rescaledX = scaler.transform(X_train)
neighbors = [1,3,5,7,9,11,13,15,17,19,21]
leaf_size=[1,2,3,5]
weights=['uniform', 'distance']
param_grid = dict(n_neighbors=neighbors,leaf_size=leaf_size,weights=weights)
model = KNeighborsClassifier()
kfold = KFold(n_splits=num_folds, random_state=seed)
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring=scoring, cv=kfold)
grid_result = grid.fit(rescaledX, Y_train)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
# Tune scaled Logistic
scaler = StandardScaler().fit(X_train)
rescaledX = scaler.transform(X_train)
c_values = [0.1, 0.3, 0.5, 0.7, 0.9, 1.0, 1.3, 1.5, 1.7, 2.0]
solver_values = ['newton-cg', 'lbfgs', 'liblinear']
param_grid = dict(C=c_values, solver=solver_values)
model = LogisticRegression()
kfold = KFold(n_splits=num_folds, random_state=seed)
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring=scoring, cv=kfold)
grid_result = grid.fit(rescaledX, Y_train)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
# Tune scaled LinearDiscriminantAnalysis
scaler = StandardScaler().fit(X_train)
rescaledX = scaler.transform(X_train)
#c_values = [0.1, 0.3, 0.5, 0.7, 0.9, 1.0, 1.3, 1.5, 1.7, 2.0]
solver_values = ['svd', 'lsqr', 'eigen']
param_grid = dict(solver=solver_values)
model = LinearDiscriminantAnalysis()
kfold = KFold(n_splits=num_folds, random_state=seed)
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring=scoring, cv=kfold)
grid_result = grid.fit(rescaledX, Y_train)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
# Tune scaled DecisionTreeClassifier
scaler = StandardScaler().fit(X_train)
rescaledX = scaler.transform(X_train)
max_features= ['auto', 'sqrt', 'log2']
min_samples_split = [2,3,4,5,6,7,8,9,10,11,12,13,14,15]
min_samples_leaf= [1,2,3,4,5,6,7,8,9,10,11]
param_grid = dict(max_features=max_features,min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf)
model = DecisionTreeClassifier(random_state=seed)
kfold = KFold(n_splits=num_folds, random_state=seed)
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring=scoring, cv=kfold)
grid_result = grid.fit(rescaledX, Y_train)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
# Tune scaled GaussianNB
scaler = StandardScaler().fit(X_train)
rescaledX = scaler.transform(X_train)
#var_smoothing_values = ['1e-09']
#priors_values = ['None']
param_grid = dict()
model = GaussianNB()
kfold = KFold(n_splits=num_folds, random_state=seed)
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring=scoring, cv=kfold)
grid_result = grid.fit(rescaledX, Y_train)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# ENSEMBLES

In [ ]:
# ensembles1
ensembles = []
ensembles.append(('AB', AdaBoostClassifier()))
ensembles.append(('GBM', GradientBoostingClassifier()))
ensembles.append(('RF', RandomForestClassifier()))
ensembles.append(('ET', ExtraTreesClassifier()))
results = []
names = []
for name, model in ensembles:
    kfold = KFold(n_splits=num_folds, random_state=seed)
    cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

In [ ]:
# Compare Algorithms
fig = pyplot.figure()
fig.suptitle('Ensemble Algorithm Comparison')
ax = fig.add_subplot(111)
pyplot.boxplot(results)
ax.set_xticklabels(names)
pyplot.show()


In [ ]:
# Tune scaled AdaBoostClassifier()
scaler = StandardScaler().fit(X_train)
rescaledX = scaler.transform(X_train)
n_estimators=[50,100,150]
learning_rate=[0.1,0.01,0.001,1.0]
param_grid = dict(n_estimators=n_estimators,learning_rate=learning_rate)
model =AdaBoostClassifier(random_state=seed)
kfold = KFold(n_splits=num_folds, random_state=seed)
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring=scoring, cv=kfold)
grid_result = grid.fit(rescaledX, Y_train)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
# Tune scaled GBM
scaler = StandardScaler().fit(X_train)
rescaledX = scaler.transform(X_train)
param_grid = dict(n_estimators=numpy.array([50,100,150,200,250,300,350,400]))
model = model = GradientBoostingClassifier(random_state=seed)
kfold = KFold(n_splits=num_folds, random_state=seed)
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring=scoring, cv=kfold)
grid_result = grid.fit(rescaledX, Y_train)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
# Tune scaled Randomforest
scaler = StandardScaler().fit(X_train)
rescaledX = scaler.transform(X_train)
criterion=['gini','entropy']
min_samples_split = [2,3,4,5,6,7,8,9,10,11,12,13,14,15]
min_samples_leaf= [1,2,3,4,5,6,7,8,9,10,11]
n_estimators=[10,15,20,25,30]
param_grid = dict(criterion=criterion,min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf,n_estimators=n_estimators)
model = RandomForestClassifier(random_state=seed)
kfold = KFold(n_splits=num_folds, random_state=seed)
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring=scoring, cv=kfold)
grid_result = grid.fit(rescaledX, Y_train)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
# Tune scaled ExtraTreesClassifier
scaler = StandardScaler().fit(X_train)
rescaledX = scaler.transform(X_train)

min_samples_split = [2,3,4,5,6,7,8,9,10,11,12,13,14,15]
min_samples_leaf= [1,2,3,4,5,6,7,8,9,10,11]

param_grid = dict(min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf)
model = ExtraTreesClassifier(random_state=seed)
kfold = KFold(n_splits=num_folds, random_state=seed)
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring=scoring, cv=kfold)
grid_result = grid.fit(rescaledX, Y_train)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
# ensembles2
ensembles = []
ensembles.append(('LGBMC', LGBMClassifier()))
ensembles.append(('XGBC', XGBClassifier()))
ensembles.append(('Cat', CatBoostClassifier()))
results = []
names = []
for name, model in ensembles:
    kfold = KFold(n_splits=num_folds, random_state=seed)
    cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

In [ ]:
# Tune scaled LGBMClassifier
scaler = StandardScaler().fit(X_train)
rescaledX = scaler.transform(X_train)
n_estimators=[50,100,150]
boosting_type=['gbdt','dart','goss','rf']
param_grid = dict(n_estimators=n_estimators,boosting_type=boosting_type)
model = LGBMClassifier(random_state=seed)
kfold = KFold(n_splits=num_folds, random_state=seed)
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring=scoring, cv=kfold)
grid_result = grid.fit(rescaledX, Y_train)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
# Tune scaled XGBClassifier
scaler = StandardScaler().fit(X_train)
rescaledX = scaler.transform(X_train)
learning_rate=[0.1,0.01,0.001,0.0001]
n_estimators=[100,150,200,250]
max_depth=[2,3,4,5,6,7,8,9,10]
param_grid = dict(learning_rate=learning_rate, n_estimators=n_estimators,max_depth=max_depth)
model = XGBClassifier(random_state=seed)
kfold = KFold(n_splits=num_folds, random_state=seed)
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring=scoring, cv=kfold)
grid_result = grid.fit(rescaledX, Y_train)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
# Tune scaled CatBoostClassifier
scaler = StandardScaler().fit(X_train)
rescaledX = scaler.transform(X_train)
learning_rate=[0.1,0.01,0.001,0.0001]
boosting_type=['gbdt','dart','goss','rf']
param_grid = dict(learning_rate=learning_rate, boosting_type=boosting_type)
model = CatBoostClassifier(random_state=seed)
kfold = KFold(n_splits=num_folds, random_state=seed)
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring=scoring, cv=kfold)
grid_result = grid.fit(rescaledX, Y_train)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# PREPARE FINAL MODEL

In [ ]:
#From overampling/undersampling for unseen dataset
X_validation = unseen.iloc[:,0:19].values #independent columns
Y_validation = unseen.iloc[:,-1].values #target column 

In [ ]:
# prepare the model
scaler = StandardScaler().fit(X_train)
rescaledX = scaler.transform(X_train)
model = SVC(C=1.5)
model.fit(rescaledX, Y_train)
# estimate accuracy on validation dataset
rescaledValidationX = scaler.transform(X_validation)
predictions = model.predict(rescaledValidationX)
print(accuracy_score(Y_validation, predictions))
print(confusion_matrix(Y_validation, predictions))
print(classification_report(Y_validation, predictions))
print(roc_auc_score(Y_validation,predictions))

# deploy the model

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()

model.fit(X, y)

pickle.dump(model, open('model.pkl','wb'))

loaded_model = pickle.load(open('model.pkl','rb'))
print(loaded_model.predict([[4, 300, 500]]))

In [ ]:
# Fit the model on 33%
model = LogisticRegression()
model.fit(X_train, Y_train)
#-------------------------
# save the model to disk
filename = 'finalized_model.sav'
dump(model, open(filename, 'wb'))
#--------------------------
# some time later...
# load the model from disk
loaded_model = load(open(filename, 'rb'))
result = loaded_model.score(X_test, Y_test)
print(result)


# FOR MEDICAL RECORD

In [ ]:
Y_validation=Y_validation.astype(int)
predictions=predictions.astype(int)

In [ ]:
# print the truth and predicted values
print ('True', Y_validation[0:20])
print ('pred', predictions[0:20])

# Adjusting the classification threshold

In [ ]:
model.predict(rescaledValidationX)[0:10] #print the first 10 predicited response
#------------------------------------
model.predict_proba(rescaledValidationX)[0:10,:] #print the first 10 predicted probabilities response for the class memeber
#-------------------------------------
model.predict_proba(rescaledValidationX)[0:10,1]  #print the first 10 predicted probabilities of one
#-------------------------------------
y_pred_prob=model.predict_proba(rescaledValidationX)[:,1]  #store the predicted probabilites for class one

# plot the probabilities values

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.show()

In [ ]:
plt.hist(y_pred_prob,bins=8,  facecolor="blue", alpha=0.5)
plt.xlim(0,1)
plt.xlabel("predicted probablities of diabetes")
plt.ylabel("frequency")
plt.title("Histogram of predicited probabilities")
plt.show();

# Decrease the threshold for predicting diabetes in order to increase the sensitivity of the classifier

In [ ]:
y_pred_prob=model.predict_proba(rescaledValidationX)[:,1]
y_pred_prob=y_pred_prob.reshape(1,-1)
y_pred_prob

In [ ]:
# predict diabetes if the predicted probabilities is greater than 0.3

from sklearn.preprocessing import binarize
y_pred_class=binarize(y_pred_prob,0.30)[0]

# prepare the final threshold model

In [ ]:
# prepare the model
scaler = StandardScaler().fit(X_train)
rescaledX = scaler.transform(X_train)
model = LogisticRegression(C=0.1, solver='newton-cg')
model.fit(rescaledX, Y_train)
# estimate accuracy on validation dataset
rescaledValidationX = scaler.transform(X_validation)
predictions = model.predict(rescaledValidationX)
print(accuracy_score(Y_validation, y_pred_class))
print(confusion_matrix(Y_validation, y_pred_class))
print(classification_report(Y_validation,y_pred_class))
print(roc_auc_score(Y_validation,y_pred_class))

# convert and print the resulted predicted class/values

In [ ]:
Y_validation=Y_validation.astype(int)
predictions=predictions.astype(int)
y_pred_class=y_pred_class.astype(int)

In [ ]:
print ( Y_validation[0:20],'true value')
print ( predictions[0:20],'predited value')
print(y_pred_class[0:20],'value after using threshold')

# print the confusion matrix for both

In [ ]:
print((""))
print(("confusion matrix without using threshold 0.25"))
print(confusion_matrix(Y_validation, predictions))
print((""))
print(("confusion matrix  using threshold 0.25"))
print(confusion_matrix(Y_validation, y_pred_class))